#Warmup
Import the numpy, pandas, matplotlib and pyspark packages in the cell below.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession

#Exercise
We are going to be using data on Student Performance during exams. Obtain the data from the following url and set up a Schema for it which you will load into pySpark. https://www.kaggle.com/spscientist/students-performance-in-exams. Set up the PySpark object and create a DataFrame on PySpark for the dataset.

In [5]:
my_spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d spscientist/students-performance-in-exams

In [ ]:
from zipfile import ZipFile
file_name = "students-performance-in-exams.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

In [11]:
from pyspark.sql.types import *

data_schema = [
               StructField('gender', StringType(), True),
               StructField('race/ethnicity', StringType(), True),
               StructField('parental level of education', StringType(), True),
               StructField('lunch', StringType(), True),
               StructField('test preparation course', StringType(), True),
               StructField('math score', IntegerType(), True),
               StructField('reading score', IntegerType(), True),
               StructField('writing score', IntegerType(), True),
            ]

final_struc = StructType(fields = data_schema)

data = spark.read.csv('StudentsPerformance.csv',sep = ',', header = True, schema = final_struc)

In [12]:
data.show(5)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

Get an average score for each student using their math, reading and writing scores. 

In [31]:
from pyspark.sql import functions as F

marksColumns = [F.col('math score'), F.col('reading score'), F.col('writing score')]
averageFunc = sum(x for x in marksColumns)/len(marksColumns)

data.withColumn('Score Avg', averageFunc).show(5,truncate=False)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------------------+
|gender|race/ethnicity|parental level of education|lunch       |test preparation course|math score|reading score|writing score|Score Avg         |
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------------------+
|female|group B       |bachelor's degree          |standard    |none                   |72        |72           |74           |72.66666666666667 |
|female|group C       |some college               |standard    |completed              |69        |90           |88           |82.33333333333333 |
|female|group B       |master's degree            |standard    |none                   |90        |95           |93           |92.66666666666667 |
|male  |group A       |associate's degree         |free/reduced|none                   |47        |57           |44   

Get a Count of the Number of Boys and girls in the class

In [27]:
data.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|  518|
|  male|  482|
+------+-----+



Apply the rank function over the overall average score that you had calculated previously.

In [32]:
data.withColumn('Score Avg', averageFunc).orderBy(F.col('Score Avg').desc()).show(5,truncate=False)

+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+-----------------+
|gender|race/ethnicity|parental level of education|lunch   |test preparation course|math score|reading score|writing score|Score Avg        |
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+-----------------+
|male  |group E       |bachelor's degree          |standard|completed              |100       |100          |100          |100.0            |
|female|group E       |bachelor's degree          |standard|none                   |100       |100          |100          |100.0            |
|female|group E       |associate's degree         |standard|none                   |100       |100          |100          |100.0            |
|female|group E       |bachelor's degree          |standard|completed              |99        |100          |100          |99.66666666666667|
|femal

Do the same exercises above but using a Pandas DataFrame instead.

In [17]:
data_pd = pd.read_csv('StudentsPerformance.csv')
data_pd.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [24]:
data_pd.groupby('gender').count().loc[:,'lunch']

gender
female    518
male      482
Name: lunch, dtype: int64

In [19]:
columns = ['math score', 'reading score', 'writing score']
data_pd['score avg'] = sum(data_pd[x] for x in columns)/len(columns)
data_pd.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,score avg
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333


In [26]:
data_pd = data_pd.sort_values(by="score avg", ascending=False)
data_pd.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,score avg
916,male,group E,bachelor's degree,standard,completed,100,100,100,100.000000
458,female,group E,bachelor's degree,standard,none,100,100,100,100.000000
962,female,group E,associate's degree,standard,none,100,100,100,100.000000
114,female,group E,bachelor's degree,standard,completed,99,100,100,99.666667
179,female,group D,some high school,standard,completed,97,100,100,99.000000


#Python Practice Question
Concordia University has the following grading policy:

Every student receives a grade in the inclusive range from 0 to 100 . Any less than 40 is a failing grade.

Sam is a professor at the university and likes to round each student's grade according to these rules: If the difference between the grade and the next multiple of 5 is less than 3, round up to the next multiple of 5. If the value of the grade is less than 38, no rounding occurs as the result will still be a failing grade. For example, grade = 84 will be rounded to 85 but grade = 29 will not be rounded because the rounding would result in a number that is less than 40.

Given the initial value of grade for each of Sam's students, write code to automate the rounding process.
Examples



```
round to  (85 - 84 is less than 3)
 do not round (result is less than 40)
 do not round (60 - 57 is 3 or higher)
```




```
Sample Input
0
4
73
67
38
33

Sample Output 
0
4
75
67
40
33
```





In [4]:
def rounding(scores):
  rounded = []
  for score in scores:
    if (score < 38) or (score%5 <= 2) :
      rounded.append(score)
    else:
      rounded.append((score//5+1)*5)
  return rounded

scores = [0,4,73,67,38,33]
print(rounding(scores))



[0, 4, 75, 67, 40, 33]
